In [17]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.postprocess import *
from src.eval import *
gezi.init_flags()

In [18]:

V = 16
models = [
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5',
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5.a',
  'deberta-v3-base.len1280.flag-encode_targets-cls.lr-5e-5',
  'deberta-v3.len1280.flag-encode_targets-cls-crank',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.ft2',
  'deberta-v3.len1280.flag-encode_targets-cls',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-nl',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-de',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-pt',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-af',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cn',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ru',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fi',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-sv',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ja',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ko',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-el',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-hr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cy',
  
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cs',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-be',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ar',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-es',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-it',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-da',
]
len(models)

20

In [19]:
oofs = [gezi.read_oof(model, V, return_dict=False) for model in models]

In [20]:
# svr_oof = oofs[0].copy()

In [21]:
# svr_pred = np.load('../working/svr.npy')
# svr_oof['pred'] = svr_pred

In [22]:
ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
# ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
# ensembler.add(oof2, weight=np.array([1,1,1,1,1,1], float))
for i, oof in enumerate(oofs):
  if i < 1:
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i < 2: 
    ensembler.add(oof, weight=np.array([1,1,10,1,1,1], float) * 2)
  elif i < 3: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.25)
  elif i < 4: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i < 5: 
    ensembler.add(oof, weight=np.array([2,1,10,1,1,2], float) * 2)
  elif i < 6: 
    ensembler.add(oof, weight=np.array([1,1,1,1,10,1], float) * 0.25)
  else:
    ensembler.add(oof, weight=np.array([1,1,10,1,0.25,0.25], float) * 0.8)

x = ensembler.finalize()
# ensembler2 = gezi.Ensembler(includes=['pred'], inplace=False)
# ensembler2.add(x, weight=np.array([1,1,1,1,1,1], float))
# # ensembler2.add(svr_oof, weight=np.array([1,1,1,1,1,1], float))
# x = ensembler2.finalize()

In [23]:
x['label'] = x.label.apply(list)
x['pred'] = x.pred.apply(list)

In [24]:
calc_metrics(np.array(list(x.label.values)), np.array(list(x.pred.values)))

OrderedDict([('score', 0.44537770120000864),
             ('score/cohesion', 0.47153674423968944),
             ('score/syntax', 0.44014063297252803),
             ('score/vocabulary', 0.40868767999126343),
             ('score/phraseology', 0.4512027723846157),
             ('score/grammar', 0.46323457415062086),
             ('score/conventions', 0.4374638034613347),
             ('score/cls', 0.4673957594647542),
             ('acc', 0.4254240177277764),
             ('acc/cohesion', 0.3891587829199693),
             ('acc/syntax', 0.42674507798517003),
             ('acc/vocabulary', 0.4627972385579136),
             ('acc/phraseology', 0.4229097417540271),
             ('acc/grammar', 0.414983380209665),
             ('acc/conventions', 0.43594988493991305),
             ('score/max', 0.5661407472757343),
             ('acc/max', 0.28713883917156735),
             ('score/min', 0.3729002606165341),
             ('acc/min', 0.4952697519815904),
             ('score/max2', 0.4454176